# Laboratoire 4 : Développement d'un système intelligent
#### Département du génie logiciel et des technologies de l’information

| Étudiants             |                                                         |
|-----------------------|---------------------------------------------------------|
| Jean-Philippe Decoste |  DECJ19059105                                           |
| Ahmad Al-Taher        |   ALTA22109307                                          |
| Stéphanie Lacerte     |   LACS06629109                                          |
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2018                                            |
| Groupe                | 2                                                       |
| Numéro du laboratoire | 04                                                      |
| Professeur            | Hervé Lombaert                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 13 dev 2018                                             |

In [1]:
from algos import DecisionTree
from algos import Knn
from algos import Bayes
from algos import NeuralNetwork
from algos import SVM
from helpers import datasets as Data
from helpers import utilities as Utils
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Using TensorFlow backend.


## Paramètres

In [2]:
# Music CSV
JMIRMFCC_CSV = r"music\tagged_feature_sets\msd-jmirmfccs_dev\msd-jmirmfccs_dev.csv"
MARSYAS_CSV = r"music\tagged_feature_sets\msd-marsyas_dev_new\msd-marsyas_dev_new.csv"
RYTHMHISTOGRAM_CSV = r"music\tagged_feature_sets\msd-rh_dev_new\msd-rh_dev_new.csv"

UNTAGGED_JMIRMFCC_CSV = r"music\untagged_feature_sets\msd-jmirmfccs_test_nolabels\msd-jmirmfccs_test_nolabels.csv"
UNTAGGED_MARSYAS_CSV = r"music\untagged_feature_sets\msd-marsyas_test_new_nolabels\msd-marsyas_test_new_nolabels.csv"
UNTAGGED_RYTHMHISTOGRAM_CSV = r"music\untagged_feature_sets\msd-rh_test_nolabels\msd-rh_test_nolabels.csv"


BASELINE_CLASSIFIER = None

# Préparation des fichiers de données

In [3]:
#1.A Prepare datasets
print("PREPARING DATASETS")
features1, answers1 = Data.prepareTrainDataset("Jmir MFCC", JMIRMFCC_CSV, True)
features1_train, answers1_train, features1_test, answers1_test, train_indexes, test_indexes = Data.splitDataSet(features1, answers1, 2, 0.8, 0)

features2, answers2 = Data.prepareTrainDataset("Marsyas", MARSYAS_CSV, True)
#features2_train, answers2_train, features2_test, answers2_test = Data.splitDataSet(features2, answers2, 2, 0.8, 0)
trian_size = len(train_indexes)
#slicing to keep only 20%
test_indexes = test_indexes[0:trian_size]

features2_train = features2[train_indexes]
answers2_train = answers2[train_indexes]
features2_test = features2[test_indexes]
answers2_test = answers2[test_indexes]

#features3, answers3 = Data.prepareTrainDataset("Rythm Histogram", RYTHMHISTOGRAM_CSV, True)
#features3_train, answers3_train, features3_test, answers3_test = Data.splitDataSet(features3, answers3, 2, 0.8, 0)
#features3_train = features3[train_indexes]
#answers3_train = answers3[train_indexes]
#features3_test = features3[test_indexes]
#answers3_test = answers3[test_indexes]


PREPARING DATASETS
Reading Jmir MFCC features:
Progress |**************************************************| 100.0% Complete
Scaling data...
Reducing dimension...
Encoding labels...

-> Done

Splitting Dataset according to these params:
Property        Value
------------  -------
n_splits          2
test_size         0.8
random_state      0
-> Done


Reading Marsyas features:
Progress |**************************************************| 100.0% Complete
Scaling data...
Reducing dimension...
Encoding labels...

-> Done

Reading Rythm Histogram features:
Progress |**************************************************| 100.0% Complete
Scaling data...
Reducing dimension...
Encoding labels...

-> Done



## Évaluation du classificateur de base

Les cinq algorithmes vue pendant les laboratoires sont prêt à être testés. Simplement l'ajouter dans la liste et la transmettre à l'analyseur. 
- Arbre de décision : <i>decisionTree</i>
- Plus proche voisin : <i>knn</i>
- Bayes : <i>bayes</i>
- Réseau de neurones : <i>neuralNetwork</i>
- Machine à vecteur : <i>SVM</i>

In [7]:
#2.A Find best algo for specified dataset
algo_to_test = ['decisionTree', 'knn', 'neuralNetwork']
best_data1 = Utils.findBaselineClassifier(algo_to_test, features1_train, answers1_train)
best_data2 = Utils.findBaselineClassifier(algo_to_test, features2_train, answers2_train)
best_data3 = Utils.findBaselineClassifier(algo_to_test, features3_train, answers3_train)
#2.B Find best algo as Baseline Classifier
BASELINE_CLASSIFIER = Utils.getBestModel([best_data1, best_data2, best_data3])
print("The baseline Classifier is:")
print(BASELINE_CLASSIFIER.getDefinition())

Train following algo to find baseLine: 
['decisionTree', 'knn', 'neuralNetwork']

New Decision Tree Classifier
1.Evaluation 

  Depth    Accuracy
-------  ----------
      5       14.42
     10       15.62
     15       13.23
The best parameters are  {'max_depth': 10}  with a score of 15.62%

-> Done



New Decision Tree Classifier
1.Training 

  Depth    Accuracy
-------  ----------
     10       26.06

-> Done



New Knn Classifier
1.Evaluation 

Weights      K    Accuracy
---------  ---  ----------
distance     5       16.95
distance    10       18.31
distance    20       19.34
The best parameters are  {'n_neighbors': 20, 'algorithm': 'auto', 'weights': 'distance'}  with a score of 19.34%

-> Done



New Knn Classifier
1.Training 

Weights      K    Accuracy
---------  ---  ----------
distance    20       99.94

-> Done



New Neural Network Classifier
1.Evaluation

  Epoch    Batch Size    Accuracy
-------  ------------  ----------
    250          1000       96.01
    800         

## Main

In [10]:
#3.B Prepare algo combination
algo1 = NeuralNetwork.neuralNetwork(2, 2, [25, 30, 15], 0.01, len(features2[0]), ['relu', 'softmax'], 800, 1000)
algo2 = Knn.knn(5, 20, 'distance')
algo3 = Knn.knn(5, 20, 'distance')

model1 = algo1.train(features2_train, answers2_train)
model2 = algo2.train(features2_train, answers2_train)
model3 = algo3.train(features1_train, answers1_train)
print("finished training")
print("start combining")
#combine
predit1 = model1.predict(features2_test)
predit2 = model2.predict(features2_test)
predit3 = model3.predict(features1_test)

#print("P1 %d; P2 %d; P3 %d" %(predit1,predit2,predit3))
predictedAnswers = []
#
for index in range(0,len(predit1)):
    if predit1[index] == predit2[index] or predit1 [index]== predit3[index]:
        predictedAnswers.append(predit1[index])
    elif predit2[index] == predit3[index]:
        predictedAnswers.append(predit2[index])
    else:
        #base on best score
        predictedAnswers.append(predit1[index])
#compute accuracy
right = 0
for index in range(0,len(predictedAnswers)):
    assert(answers1_test[index] == answers2_test[index])
    if predictedAnswers[index] == answers1_test[index]:
        right += 1
accuracy = right / len(features2_test) *100

print("We got %d right answers" % (right))
print("Précision %.2f"% (accuracy))

#transform back the answers from numeric to label
'''answers = []
for index in range(0,len(ids)):
    answers.append(random.randint(1,14))
labels = labelEncoder.inverse_transform(answers)
Data.outPut(ids,labels)'''


New Neural Network Classifier

New Knn Classifier

New Knn Classifier
1.Training
  Epoch    Batch Size    Accuracy
-------  ------------  ----------
    800          1000       96.09

1.Training 

Weights      K    Accuracy
---------  ---  ----------
distance    20       99.94

-> Done


1.Training 

Weights      K    Accuracy
---------  ---  ----------
distance    20       99.94

-> Done


finished training
start combining
We got 7932 right answers
Précision 22.09


## Introduction et revue de la littérature (Question 1)

Plusieurs techniques sont en places depuis quelques années déjà afin de classifier les différents sons et musique. Des descripteurs cepstraux sont souvent utilisés pour le traitement de la parole par exemple. Selon la thèse produite par Slim Essid portant sur la classification automatique des signaux audiofréquences : reconnaissance des instruments de musique, la représentation cepstrale s’avère efficace pour de nombreuses tâches de classification audio telles que la discrimination parole/musique, la reconnaissance du genre ou encore la reconnaissance des instruments.

Dans le cadre du laboratoire, les Mel-Frequency Cepstral Coefficients (MFCC) sont justement sujets à classification. Mentionné dans plusieurs tel que Pachet - Cazaly en 2000, McKay et Fujinaga en 2004 ainsi que Li et Ogihara en 2005, la classification, spécifiquement hiérarchique a eu un grand succès pour les tâches de classification audio et du genre musical. La classification hiérarchique a pour but d'améliorer les taux de reconnaissance par rapport à ceux obtenus avec les systèmes dits “plats”, dans lesquels toutes les classes sont considérées sur un seul niveau, sans organisation particulière. De plus, l’application de regroupement (clustering) hiérarchique a également fait ses preuves pour la classification d’instruments non connus à l'étape d’apprentissage. La distance, tel que vu avec le k-mean dans le cours sur le regroupement, est utilisée comme critère de proximité des classes dans le processus de clustering, en faisant l'hypothèse de gaussianité des données.

En terme de statistique, il est possible d'observer différents résultats selon les articles lus. En effet, le taux de classification dépend des modèles utilisés et de leur primitive. Par exemple, obtenant un taux de précision d’environ 43% avec un réseau de neurones, l’article de Hu et Ogihara, en 2012 fait ces preuves face aux résultats de précision avec des classifications aléatoires. Hu et Ogihara obtiennent également un taux de classification en utilisant la combinaison de modèle à environ 84% ce qui est très acceptable.

Dans ce présent laboratoire, l’objectif premier est de construire un système intelligent permettant de classer automatiquement des échantillons de données en utilisant la combinaison de modèles et des algorithmes d’apprentissage. Le tout se fait avec les technologies Python. Dans les sections abordées ci-dessous, les sujets de configuration de la machine, l’approche de partitionnement, le choix des primitives et les modèles d’apprentissage choisi seront expliqués. De plus, les hyperparamètres seront présentés en plus de la solution finale et les améliorations possibles.

## Question 2
### Configuration de l'environnement

Dans le cadre de ce laboratoire, le projet final a été réalisé à l’aide du langage de programmation Python et Jupiter Notebook sous le système d’exploitation Windows. Les machines utilisées pour l'exécution se résument à un processeur Intel(R) Core(TM) i7-4770k CPU 3.40GHz ainsi que 16Go de mémoire vive.

### Partitionnement des données

Pour décider du classificateur de base, les algorithmes n'ont reçu que 20% des données afin d'augmenter leur exécution. Lors des tests préliminaires, ce chiffre était d'environ 50%, mais la variation du résultat d'entrainement était moindre et n'expliquait pas l'heure supplémentaire de temps de calcul.

### Fichier de primitives choisie

À ce qui a trait au choix de primitives (CSV), l'analyse des données, la lecture de la documentation fournit ainsi que plusieurs tests ont permi de déterminer que les fichiers qui permettront aux algorithmes de performer le mieux sont les suivants: JMIR MFCCs, MARSYAS ainsi que Rythm Histogram

Bien sûr, la dimension ainsi que la taille des données ont influencer cetter décision puisque le temps d'exécution des algorithmes n'est pas à négliger.

### Validation

La méthode de validation utilisée est StratifiedShuffleSplit. La performance des algorithmes avec cette méthode sur les ensembles de données est adéquate et évite des temps d’exécution trop long. Elle est, entre autre,  utilisé pour l'évaluation et l'entraînement du Baseline Classifier. La combinaison d’algorithme, quant à elle, est effectuée à l’aide de K-Fold cross-validation (5).

### Prétraitement

Afin de s'assurer d'une performance optimal, il a fallu normaliser les données en utilisant la méthode <i>MinMaxScaler</i>. Puisque le temps d'exécution était un facteur important pour l'équipe, il a également fallu réduire la dimension des données à la de PCA. De plus, KNN n'est efficace que si on lui envoi un nombre limité de dimension, cette approche offrait donc une meilleur compatibilité entre les algorithmes. Il y eu une modification supplémentaire pour optimiser le réseau neuronaux, transformer le tableau contenant les réponses en tableau binaire pour représenter chacune des classes. (EX:. avant: [4], apres: [0, 0, 0, 1])

## Question 3
### Description des modèles et justifications

## Question 4
### Matrice des expérimentations

### Matrice des résultats de l'étude des hyperparamètres

### Graphiques

### Discussion

## Question 5
### Présentation de la conception

## Question 6
### Hyperparamètres des modèles choisis dans la conception

### Matrice des résultats

### Discussion

## Question 7 
### Formulation des recommandations

## Conclusion

## Références
[Kitahara et al., 2004] 
T. Kitahara, M. Goto, et H.G. Okuno. Category-level identification of non-registered musical instrument sounds. Dans International Conference on Acoustics, Speech, and Signal Processing (ICASSP), Montreal, Canada., mai 2004.

[Pachet et Cazaly, 2000] 
F. Pachet et D. Cazaly. A taxonomy of musical genres. Dans Content-Based Multimedia Information Access Conference (RIAO), Paris, France, avril 2000.

[McKay et Fujinaga, 2004] 
C. McKay et I. Fujinaga. Automatic genre classification using large high-level musical feature sets. Dans 5th International Conference on Music Information Retrieval (ISMIR), Barcelona, Spain, octobre 2004.

[Li et Ogihara, 2005] 
Tao Li et Mitsunori Ogihara. Music genre classification with taxonomy. Dans International Conference on Acoustics, Speech, and Signal Processing (ICASSP), Philadelphia, USA, mars 2005.
